<a href="https://colab.research.google.com/github/MathewLipman/Work-Samples/blob/main/SQL_e_commerce_dataset_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
%load_ext sql
%sql sqlite:/content/drive/MyDrive/Dataset/chinook.db


In [ ]:
import sqlite3
import pandas as pd

In [ ]:
conn = sqlite3.connect('/content/drive/MyDrive/Dataset/chinook.db')

In [ ]:
q1 = '''
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");
'''
pd.read_sql_query(q1, conn)


,name,type
0,album,table
1,artist,table
2,customer,table
3,employee,table
4,genre,table
5,invoice,table
6,invoice_line,table
7,media_type,table
8,playlist,table
9,playlist_track,table


In [ ]:

q2 = '''

SELECT
    genre.name AS genre,    
    sum(invoice_line.quantity) AS tracks_sold
FROM track
INNER JOIN genre ON track.genre_id = genre.genre_id
INNER JOIN invoice_line ON track.track_id = invoice_line.track_id
GROUP BY genre.name
ORDER BY tracks_sold DESC
LIMIT 5
'''

pd.read_sql_query(q2, conn)


,genre,tracks_sold
0,Rock,2635
1,Metal,619
2,Alternative & Punk,492
3,Latin,167
4,R&B/Soul,159


In [ ]:

q3 = '''

SELECT
    genre.name AS genre,    
    sum(invoice_line.quantity) AS tracks_sold,
    sum(invoice.total) as total_sold    
FROM track
INNER JOIN genre ON track.genre_id = genre.genre_id
INNER JOIN invoice_line ON track.track_id = invoice_line.track_id
INNER JOIN invoice on invoice_line.invoice_id = invoice.invoice_id
GROUP BY genre.name
ORDER BY tracks_sold DESC
LIMIT 5
'''

pd.read_sql_query(q3, conn)


,genre,tracks_sold,total_sold
0,Rock,2635,26751.78
1,Metal,619,5316.30
2,Alternative & Punk,492,4841.10
3,Latin,167,1705.77
4,R&B/Soul,159,1751.31


In [ ]:
q4 = '''
WITH 
    total_tracks_sold AS
    (
     SELECT
       genre.name AS genre,
       sum(invoice_line.quantity) AS total_tracks_sold
     FROM track
     INNER JOIN genre ON track.genre_id = genre.genre_id
     INNER JOIN invoice_line ON track.track_id = invoice_line.track_id
     GROUP BY genre.name
    ),
    overall_tracks_sold AS
    (
     SELECT sum(total_tracks_sold) AS overall_sold
     FROM total_tracks_sold
    )

SELECT
    genre.name AS genre,
    invoice.billing_country AS country,
    sum(invoice_line.quantity) AS tracks_sold,
    (sum(invoice_line.quantity) * 1.0 / o.overall_sold) as track_percentage  
FROM track
INNER JOIN genre ON track.genre_id = genre.genre_id
INNER JOIN invoice_line ON track.track_id = invoice_line.track_id
INNER JOIN invoice on invoice_line.invoice_id = invoice.invoice_id
INNER JOIN total_tracks_sold t ON t.genre = genre.name
INNER JOIN overall_tracks_sold o
WHERE invoice.billing_country = 'USA'
GROUP BY genre.name, invoice.billing_country
ORDER BY track_percentage DESC
LIMIT 10
'''

pd.read_sql_query(q4, conn)


,genre,country,tracks_sold,track_percentage
0,Rock,USA,561,0.117931
1,Alternative & Punk,USA,130,0.027328
2,Metal,USA,124,0.026067
3,R&B/Soul,USA,53,0.011141
4,Blues,USA,36,0.007568
5,Alternative,USA,35,0.007358
6,Latin,USA,22,0.004625
7,Pop,USA,22,0.004625
8,Hip Hop/Rap,USA,20,0.004204
9,Jazz,USA,14,0.002943


In [ ]:
q4 = '''
WITH 
    total_tracks_sold AS
    (
     SELECT
       sum(invoice_line.quantity) AS total_tracks_sold
     FROM track
     INNER JOIN invoice_line ON track.track_id = invoice_line.track_id
    )

SELECT
    invoice.billing_country AS country,
    SUM(invoice_line.quantity) AS transacations,
    sum(invoice.total) AS total_billed, 
    invoice.total / sum(invoice.total) as sold_percentage,
    sum(invoice.total) / SUM(invoice_line.quantity) as avg_transacation_amount
FROM track
INNER JOIN genre ON track.genre_id = genre.genre_id
INNER JOIN invoice_line ON track.track_id = invoice_line.track_id
INNER JOIN invoice on invoice_line.invoice_id = invoice.invoice_id
INNER JOIN total_tracks_sold
GROUP BY 1 
ORDER BY 5 DESC
LIMIT 10
'''

pd.read_sql_query(q4, conn)


,country,transacations,total_billed,sold_percentage,avg_transacation_amount
0,Ireland,116,1433.52,0.007597,12.357931
1,Czech Republic,276,3183.84,0.002799,11.535652
2,Australia,82,940.50,0.002105,11.469512
3,Spain,99,1076.13,0.007360,10.870000
4,Hungary,79,830.61,0.007151,10.514051
5,India,185,1887.93,0.005244,10.205027
6,Germany,338,3441.24,0.003165,10.181183
7,Canada,541,5489.55,0.001803,10.147043
8,France,393,3972.87,0.000498,10.109084
9,United Kingdom,248,2498.76,0.003962,10.075645


In [ ]:
q6 = '''
WITH 
    total_tracks_sold AS
    (
     SELECT
       sum(invoice_line.quantity) AS total_tracks_sold
     FROM track
     INNER JOIN invoice_line ON track.track_id = invoice_line.track_id
    ),
    man_id AS
    (
      SELECT
      employee.last_name ||  ',' || employee.first_name as Employee
      FROM Employee
      WHERE employee.employee_id = 2     
    )

SELECT
    employee.last_name ||  ',' || employee.first_name as Employee,
      FROM man_id
    employee.last_name ||  ',' || employee.first_name as Employee,
    employee.title as title,
    employee.reports_to as manager,
    SUM(invoice_line.quantity) AS transacations,
    sum(invoice.total) AS total_billed, 
    invoice.total / sum(invoice.total) as sold_percentage,
    sum(invoice.total) / SUM(invoice_line.quantity) as avg_transacation_amount
FROM track
INNER JOIN genre ON track.genre_id = genre.genre_id
INNER JOIN invoice_line ON track.track_id = invoice_line.track_id
INNER JOIN invoice on invoice_line.invoice_id = invoice.invoice_id
INNER JOIN total_tracks_sold
INNER JOIN CUSTOMER on invoice.customer_id = customer.customer_id
INNER JOIN EMPLOYEE on customer.support_rep_id = employee.employee_Id
GROUP BY 1
LIMIT 10
'''

pd.read_sql_query(q6, conn)


In [ ]:
q7 = '''
WITH 
    total_tracks_sold AS
    (
     SELECT
       sum(invoice_line.quantity) AS total_tracks_sold
     FROM track
     INNER JOIN invoice_line ON track.track_id = invoice_line.track_id
    )

SELECT
    man.last_name || ',' || man.first_name as Manager,
    emp.last_name ||  ',' || emp.first_name as Employee,
    emp.title as title,
    SUM(invoice_line.quantity) AS transactions,
    sum(invoice.total) AS total_billed, 
    (invoice.total) / sum(invoice.total) as sold_percentage,
    sum(invoice.total) / SUM(invoice_line.quantity) as avg_transaction_amount
FROM track
INNER JOIN genre ON track.genre_id = genre.genre_id
INNER JOIN invoice_line ON track.track_id = invoice_line.track_id
INNER JOIN invoice on invoice_line.invoice_id = invoice.invoice_id
INNER JOIN total_tracks_sold
INNER JOIN CUSTOMER on invoice.customer_id = customer.customer_id
INNER JOIN EMPLOYEE emp on customer.support_rep_id = emp.employee_Id
INNER JOIN EMPLOYEE man on emp.reports_to = man.employee_Id
GROUP BY 2
LIMIT 10
'''

pd.read_sql_query(q7, conn)


,Manager,Employee,title,transactions,total_billed,sold_percentage,avg_transaction_amount
0,"Edwards,Nancy","Johnson,Steve",Sales Support Agent,1408,13333.32,0.000149,9.469687
1,"Edwards,Nancy","Park,Margaret",Sales Support Agent,1600,15923.16,0.000124,9.951975
2,"Edwards,Nancy","Peacock,Jane",Sales Support Agent,1749,18246.69,0.000868,10.432642


In [ ]:
q8 ='''
select
  sum(invoice.total),
  sum(invoice_line.quantity)
FROM invoice
INNER JOIN invoice_line on invoice.invoice_id = invoice_line.invoice_id
'''
pd.read_sql_query(q8, conn)

,sum(invoice.total),sum(invoice_line.quantity)
0,47503.17,4757


In [ ]:
q9 = '''
WITH 
    total_tracks_sold AS
    (
     SELECT
       sum(invoice_line.quantity) AS total_tracks_sold
     FROM track
     INNER JOIN invoice_line ON track.track_id = invoice_line.track_id
    )

SELECT
    invoice.billing_country AS country,
    count(invoice_line.quantity) AS transacations,
    sum(invoice.total) AS total_billed, 
    invoice.total / sum(invoice.total) as sold_percentage,
    sum(invoice.total) / SUM(invoice_line.quantity) as avg_transacation_amount,
    customer.customer_id as total_customers
FROM track
INNER JOIN genre ON track.genre_id = genre.genre_id
INNER JOIN invoice_line ON track.track_id = invoice_line.track_id
INNER JOIN invoice ON invoice_line.invoice_id = invoice.invoice_id
INNER JOIN total_tracks_sold
INNER JOIN customer ON invoice.customer_id = customer.customer_id
GROUP BY 1 
ORDER BY 6 DESC
LIMIT 10
'''

pd.read_sql_query(q9, conn)


,country,transacations,total_billed,sold_percentage,avg_transacation_amount,total_customers
0,India,185,1887.93,0.005244,10.205027,59
1,Chile,98,912.78,0.006508,9.314082,57
2,Argentina,40,396.00,0.022500,9.900000,56
3,Australia,82,940.50,0.002105,11.469512,55
4,United Kingdom,248,2498.76,0.003962,10.075645,53
5,Sweden,76,754.38,0.009186,9.926053,51
6,Spain,99,1076.13,0.007360,10.870000,50
7,Poland,77,690.03,0.015782,8.961429,49
8,Netherlands,66,544.50,0.021818,8.250000,48
9,Italy,51,468.27,0.004228,9.181765,47


In [262]:
q10 = '''
SELECT *
FROM track
INNER JOIN invoice_line ON track.track_id = invoice_line.track_id
INNER JOIN invoice ON invoice_line.invoice_id = invoice.invoice_id
INNER JOIN album ON track.album_id = album.album_id
ORDER BY invoice.customer_id

'''

pd.read_sql_query(q10, conn)

,track_id,name,album_id,media_type_id,genre_id,...,billing_postal_code,total,album_id,title,artist_id
0,2160,Nothingman,178,1,1,...,12227-000,8.91,178,Live On Two Legs [live],118
1,875,Acelerou,70,1,7,...,12227-000,8.91,70,Djavan Ao Vivo - Vol. 1,80
2,83,Wherever I May Roam,9,1,3,...,12227-000,8.91,9,Plays Metallica By Four Cellos,7
3,16,Dog Eat Dog,4,1,1,...,12227-000,8.91,4,Let There Be Rock,1
4,1756,Magamalabares,145,1,7,...,12227-000,8.91,145,Barulhinho Bom,103
...,...,...,...,...,...,...,...,...,...,...,...
4752,2645,Back Door Man,214,1,1,...,560001,4.95,214,The Doors,140
4753,30,Amazing,5,1,1,...,560001,4.95,5,Big Ones,3
4754,1351,Rime of the Ancient Mariner,107,1,3,...,560001,4.95,107,Powerslave,90
4755,1799,Shakes and Ladders,147,1,1,...,560001,4.95,147,The Best Of Men At Work,105


In [240]:
q11 = '''
album_purchases AS (
                    SELECT
                      invoice.invoice_id
                    FROM invoice
                    WHERE count(invoice.invoice_id) > 3
)



SELECT *
FROM track
INNER JOIN invoice_line ON track.track_id = invoice_line.track_id
INNER JOIN invoice ON invoice_line.invoice_id = invoice.invoice_id
INNER JOIN album ON track.album_id = album.album_id
GROUP BY invoice.customer_id, invoice.invoice_id, track.album_id
ORDER BY invoice.customer_id
LIMIT 20
'''

pd.read_sql_query(q11, conn)

DatabaseError: ignored

In [289]:
q12 = '''
WITH album_purchases AS (
                    SELECT
                      invoice.invoice_id
                    FROM invoice
                    INNER JOIN invoice_line ON invoice.invoice_id = invoice_line.invoice_id
                    GROUP BY invoice.invoice_id
                    HAVING count(invoice.invoice_id) > 10
)

SELECT *
FROM track
INNER JOIN invoice_line ON track.track_id = invoice_line.track_id
INNER JOIN album_purchases ON invoice_line.invoice_id = album_purchases.invoice_id
INNER JOIN invoice ON invoice_line.invoice_id = invoice.invoice_id
INNER JOIN album ON track.album_id = album.album_id
LIMIT 5
'''

pd.read_sql_query(q12, conn)


,track_id,name,album_id,media_type_id,genre_id,...,billing_postal_code,total,album_id,title,artist_id
0,1158,Right Next Door to Hell,91,2,1,...,10012-2612,15.84,91,Use Your Illusion I,88
1,1159,Dust N' Bones,91,2,1,...,10012-2612,15.84,91,Use Your Illusion I,88
2,1160,Live and Let Die,91,2,1,...,10012-2612,15.84,91,Use Your Illusion I,88
3,1161,Don't Cry (Original),91,2,1,...,10012-2612,15.84,91,Use Your Illusion I,88
4,1162,Perfect Crime,91,2,1,...,10012-2612,15.84,91,Use Your Illusion I,88


In [301]:
q13 ='''
SELECT 
COUNT(DISTINCT customer.customer_id) as total_customers,
COUNT(DISTINCT invoice.invoice_id) as total_transacations,
CAST(COUNT(DISTINCT invoice.invoice_id) AS FLOAT) / COUNT(DISTINCT customer.customer_id) as avg_trans_per_cust,
COUNT(DISTINCT invoice_line.invoice_line_id) AS total_items_sold,
CAST(COUNT(DISTINCT invoice_line.invoice_line_id) AS FLOAT) / COUNT(DISTINCT invoice.invoice_id) as avg_n_items_sold
FROM invoice
INNER JOIN customer ON invoice.customer_id = customer.customer_id     
INNER JOIN invoice_line ON invoice.invoice_id = invoice_line.invoice_id
'''
pd.read_sql_query(q13, conn)


,total_customers,total_transacations,avg_trans_per_cust,total_items_sold,avg_n_items_sold
0,59,614,10.40678,4757,7.747557
